In [1]:
import os
import math
import numpy as np
import pandas as pd
from ast import literal_eval
import itertools
import arviz as az
import scipy.stats as st
import seaborn as sns
import pymc3 as pm
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
pd.options.mode.chained_assignment = None  # default='warn'

# Data

In [4]:
data = pd.read_csv("data/soc_data_iv.csv")

In [5]:
np.mean(data[data.done == True].time_played), np.std(data[data.done == True].time_played)

(18.54107029051394, 2.7537963271560457)

In [13]:
data.columns

Index(['ID', 'level', 'trial', 'time_played', 'N_attempt', 'done',
       'expectancy', 'SoC', 'crashed', 'N_drift', 'N_prior_crashs',
       'SoC_last_trial', 'trials_since_last_crash', 'crashed_in_last_trial',
       'consecutive_crash_success', 'N_consecutive_crash_success',
       'Neuroticism', 'Extraversion', 'Openness', 'Conscientiousness',
       'Agreeableness', 'Neuroticism_norm', 'Extraversion_norm',
       'Openness_norm', 'Conscientiousness_norm', 'Agreeableness_norm'],
      dtype='object')

In [14]:
# let's annotate the data a little bit and make it more concise
df = data.dropna(subset=["ID", "expectancy", "N_drift", "SoC", "SoC_last_trial", "crashed", "crashed_in_last_trial", "N_consecutive_crash_success"]).copy()

# Encode IDs
df["ID_idx"] = pd.Categorical(df["ID"]).codes
n_participants = df["ID_idx"].nunique()

# Bayesian updating process

- we will treat SoC as normally distributed posterior.
- we will split the data by participant ID.
- for every row within the participant data we will model the Bayesian updating process.
 <br />

- we will assess the weighting of the likelihood assuming the prior weight is =1. This will tell us whether individual participants relied more on their prior or their inferred performance when rating their Sense of Control (SoC)

But to find out the difference in weighting of prior and likelihood, we first have to define prior and likelihood, that is specify what constitute them. For this, we can refer to our final selected linear mixed models.

$prior \approx N_{(crash-success)} + SoC_{t-1}$

$likelihood \approx N_{Drift} + crashed_{(0,1)} + crashed_{t-1;0,1} + N_{(crash-success)}$

## Hierarchical Bayesian Model with dynamic likelihood weighting

Let the log weight evolve linearly across trials:

$log(w){_i,_t} = \alpha{_i} + \beta{_i} * trial{_i,_t}$

In [15]:
n_participants = df["ID_idx"].nunique()

with pm.Model() as time_varying_model:
    # hyperpriors
    mu_alpha = pm.Normal("mu_alpha", 0, 1)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 1)
    
    mu_beta = pm.Normal("mu_beta", 0, 1)
    sigma_beta = pm.HalfNormal("sigma_beta", 1)

    # per-participant intercept and slope
    alpha = pm.Normal("alpha", mu=mu_alpha, sigma=sigma_alpha, shape=n_participants)
    beta = pm.Normal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_participants)

    # trial-level weight per row (indexed by participant and trial)
    trial_idxs = df["trial"].values
    participant_idxs = df["ID_idx"].values

    log_w = alpha[participant_idxs] + beta[participant_idxs] * trial_idxs
    w = pm.Deterministic("w", pm.math.exp(log_w))  # ensures w > 0
    
    # --- Linear prior: SoC_last_trial + N_consecutive_crash_success ---
    b_soc_last = pm.Normal("b_soc_last", 0, 1)
    b_n_consec = pm.Normal("b_n_consec", 0, 1)
    prior = b_soc_last * df["SoC_last_trial"].values + b_n_consec * df["N_consecutive_crash_success"].values

    # --- Linear likelihood: crashed + crashed_in_last_trial + N_drift ---
    b_crashed = pm.Normal("b_crashed", 0, 1)
    b_crashed_prev = pm.Normal("b_crashed_prev", 0, 1)
    b_drift = pm.Normal("b_drift", 0, 1)
    likelihood = (
        b_crashed * df["crashed"].values +
        b_crashed_prev * df["crashed_in_last_trial"].values +
        b_drift * df["N_drift"].values
    )
    
    # --- Bayesian update to compute SoC ---
    mu_soc = (prior + w * likelihood) / (1 + w)
    
    sigma = pm.HalfNormal("sigma", 0.1)
    SoC_obs = pm.Normal("SoC_obs", mu=mu_soc, sigma=sigma, observed=df["SoC"].values)
    
    # defining deterministic variable to keep true SoC 
    SoC_pred = pm.Deterministic("SoC_pred", mu_soc)

    trace = pm.sample(1000, tune=1000, target_accept=0.95)
    #trace = pm.sample(1000, target_accept=0.95)


/var/folders/z1/crk0j_p52yq850yynjpq93wm0000gn/T/ipykernel_6103/880337503.py:46: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, target_accept=0.95)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, b_drift, b_crashed_prev, b_crashed, b_n_consec, b_soc_last, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)


RuntimeError: Chain 0 failed.

In [21]:
with pm.Model() as time_varying_model:
    # indexing data
    trial_idxs = pm.Data("trial_idxs", df["trial"].values)
    participant_idxs = pm.Data("participant_idxs", df["ID_idx"].values)

    # hyperpriors
    mu_alpha = pm.Normal("mu_alpha", 0, 1)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 1)
    
    mu_beta = pm.Normal("mu_beta", 0, 1)
    sigma_beta = pm.HalfNormal("sigma_beta", 1)

    # per-participant intercept and slope
    alpha = pm.Normal("alpha", mu=mu_alpha, sigma=sigma_alpha, shape=n_participants)
    beta = pm.Normal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_participants)
    
    # Trial- and participant-specific weight
    log_w = alpha[participant_idxs] + beta[participant_idxs] * trial_idxs
    w = pm.Deterministic("w", pm.math.exp(log_w))

    # Observed data
    SoC_last = pm.Data("SoC_last", df["SoC_last_trial"].values)
    N_consec = pm.Data("N_consecutive", df["N_consecutive_crash_success"].values)
    crashed = pm.Data("crashed", df["crashed"].values)
    crashed_last = pm.Data("crashed_last", df["crashed_in_last_trial"].values)
    N_drift = pm.Data("N_drift", df["N_drift"].values)

    prior = SoC_last + N_consec
    likelihood = crashed + crashed_last + N_drift

    mu_soc = (prior + w * likelihood) / (1 + w)
    
    sigma = pm.HalfNormal("sigma", 0.1)
    SoC_obs = pm.Normal("SoC_obs", mu=mu_soc, sigma=sigma, observed=df["SoC"].values)

    SoC_pred = pm.Deterministic("SoC_pred", mu_soc)

    trace = pm.sample(1000, tune=1000, target_accept=0.95)


/var/folders/z1/crk0j_p52yq850yynjpq93wm0000gn/T/ipykernel_6103/3487145949.py:38: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, target_accept=0.95)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/pymc3/step_methods/hmc/quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)


RuntimeError: Chain 2 failed.

In [24]:
with pm.Model() as simple_model:
    # Indexing data
    trial_idxs = pm.Data("trial_idxs", df["trial"].values)
    participant_idxs = pm.Data("participant_idxs", df["ID_idx"].values)

    # Inputs
    expectancy = pm.Data("expectancy", df["expectancy"].values)
    N_drift = pm.Data("N_drift", df["N_drift"].values)

    # Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", 0, 1)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 1)
    
    mu_beta = pm.Normal("mu_beta", 0, 1)
    sigma_beta = pm.HalfNormal("sigma_beta", 1)

    # Per-participant parameters
    alpha = pm.Normal("alpha", mu=mu_alpha, sigma=sigma_alpha, shape=n_participants)
    beta = pm.Normal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_participants)

    # Weight (log scale to keep positive)
    log_w = alpha[participant_idxs] + beta[participant_idxs] * trial_idxs
    w = pm.Deterministic("w", pm.math.exp(log_w))

    # Weighted average of prior and likelihood
    mu_soc = (expectancy + w * N_drift) / (1 + w)

    # Observation model
    sigma = pm.HalfNormal("sigma", 0.1)
    SoC_obs = pm.Normal("SoC_obs", mu=mu_soc, sigma=sigma, observed=df["SoC"].values)

    # Predicted value
    SoC_pred = pm.Deterministic("SoC_pred", mu_soc)

    # Sampling
    trace = pm.sample(1000, tune=1000, target_accept=0.95)


/var/folders/z1/crk0j_p52yq850yynjpq93wm0000gn/T/ipykernel_6103/1440963533.py:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, target_accept=0.95)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/heinrich/anaconda3/envs/datascience/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 162 seconds.


In [25]:
with pm.Model() as expanded_prior_model:
    
    # Indexing
    trial_idxs = pm.Data("trial_idxs", df["trial"].values)
    participant_idxs = pm.Data("participant_idxs", df["ID_idx"].values)

    # Inputs
    SoC_last = pm.Data("SoC_last_trial", df["SoC_last_trial"].values)
    N_crash_success = pm.Data("N_consecutive_crash_success", df["N_consecutive_crash_success"].values)
    N_drift = pm.Data("N_drift", df["N_drift"].values)

    # Hyperpriors for prior weights (SoC_last and crash/success history)
    mu_alpha = pm.Normal("mu_alpha", 0, 1, shape=2)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 1, shape=2)

    # Participant-level coefficients for prior
    alpha = pm.Normal("alpha", mu=mu_alpha, sigma=sigma_alpha, shape=(n_participants, 2))

    # Compute prior: linear combo of predictors
    prior = (
        alpha[participant_idxs, 0] * SoC_last +
        alpha[participant_idxs, 1] * N_crash_success
    )

    # Hyperpriors for drift weighting
    mu_beta = pm.Normal("mu_beta", 0, 1)
    sigma_beta = pm.HalfNormal("sigma_beta", 1)

    beta = pm.Normal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_participants)

    # Weight (log scale)
    log_w = beta[participant_idxs] * trial_idxs
    w = pm.Deterministic("w", pm.math.exp(log_w))

    # Posterior expectation (weighted average of prior and likelihood)
    mu_soc = (prior + w * N_drift) / (1 + w)

    # Likelihood
    sigma = pm.HalfNormal("sigma", 0.1)
    SoC_obs = pm.Normal("SoC_obs", mu=mu_soc, sigma=sigma, observed=df["SoC"].values)

    # Predictions
    SoC_pred = pm.Deterministic("SoC_pred", mu_soc)

    # Sampling
    trace = pm.sample(1000, tune=1000, target_accept=0.95)


/var/folders/z1/crk0j_p52yq850yynjpq93wm0000gn/T/ipykernel_6103/1673918473.py:46: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, target_accept=0.95)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, sigma_beta, mu_beta, alpha, sigma_alpha, mu_alpha]


Sampling 1 chain for 1_000 tune and 197 draw iterations (1_000 + 197 draws total) took 537 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks
